In [ ]:
from importlib.metadata import version
import tiktoken
print("tiktoken version:", version("tiktoken"))

In [ ]:
tokenizer = tiktoken.get_encoding("gpt2")

In [ ]:
text = ("Hello, do you like tea? <|endoftext|> In the sunlit terraces""of someunknownPlace."

In [ ]:
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

In [ ]:
strings = tokenizer.decode(integers)
print(strings)